In [2]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import os
import h5py
import ecg_plot
import matplotlib.pyplot as plt
from openpyxl.worksheet.datavalidation import DataValidation

order = [
    'SB', 'SB', 'SB', 'SB', 'SB',
    'SB', 'AF', 'SB', 'AF','SB', 'AF','SB', 'AF','SB', 'AF',
    'SB', 'AF', 'LBBB', 'SB', 'AF', 'LBBB','SB', 'AF', 'LBBB',
    'SB', 'AF', 'LBBB', 'RBBB' , 'SB', 'AF', 'LBBB', 'RBBB' , 'SB', 'AF', 'LBBB', 'RBBB' , 'SB', 'AF', 'LBBB', 'RBBB' , 
    'SB', 'AF', 'LBBB', 'RBBB' , 'SB', 'AF', 'LBBB', 'RBBB' , 'SB', 'AF', 'LBBB', 'RBBB' , 'SB', 'AF', 'LBBB', 'RBBB' , 
    'SB', 'AF', 'LBBB', 'RBBB' , 'SB', 'AF', 'LBBB', 'RBBB' , 'SB', 'AF', 'LBBB', 'RBBB' , 'SB', 'AF', 'LBBB', 'RBBB' , 
    'SB', 'AF', 'LBBB', 'RBBB' , 'ST', 'SB', 'AF', 'LBBB', 'RBBB' , 'ST', 'SB', 'AF', 'LBBB', 'RBBB' , 'ST', 'SB', 'AF', 'LBBB', 'RBBB' , 'ST', 
    'SB', 'AF', 'LBBB', 'RBBB' , 'ST', 'SB', 'AF', 'LBBB', 'RBBB' , 'ST', 'SB', 'AF', 'LBBB', 'RBBB' , 'ST', 'SB', 'AF', 'LBBB', 'RBBB' , 'ST', 
    'ST', 'SB', 'AF', 'LBBB', 'RBBB' , 'ST', 
]

In [3]:
import pandas as pd
from collections import Counter

# Carregar os DataFrames das amostras
SB = pd.read_csv('SB_amostras.csv')
ST = pd.read_csv('ST_amostras.csv')
AF = pd.read_csv('AF_amostras.csv')
RBBB = pd.read_csv('RBBB_amostras.csv')
LBBB = pd.read_csv('LBBB_amostras.csv')

# Definir a ordem desejada dos DataFrames
order = [
    'SB', 'SB', 'SB', 'SB', 'SB',
    'SB', 'AF', 'SB', 'AF', 'SB', 'AF', 'SB', 'AF', 'SB', 'AF',
    'SB', 'AF', 'LBBB', 'SB', 'AF', 'LBBB', 'SB', 'AF', 'LBBB',
    'SB', 'AF', 'LBBB', 'RBBB', 'SB', 'AF', 'LBBB', 'RBBB', 'SB', 'AF', 'LBBB', 'RBBB',
    'SB', 'AF', 'LBBB', 'RBBB', 'SB', 'AF', 'LBBB', 'RBBB', 'SB', 'AF', 'LBBB', 'RBBB',
    'SB', 'AF', 'LBBB', 'RBBB', 'SB', 'AF', 'LBBB', 'RBBB', 'SB', 'AF', 'LBBB', 'RBBB',
    'SB', 'AF', 'LBBB', 'RBBB', 'SB', 'AF', 'LBBB', 'RBBB', 'SB', 'AF', 'LBBB', 'RBBB',
    'ST', 'SB', 'AF', 'LBBB', 'RBBB', 'ST', 'SB', 'AF', 'LBBB', 'RBBB', 'ST', 'SB', 'AF', 'LBBB', 'RBBB',
    'ST', 'SB', 'AF', 'LBBB', 'RBBB', 'ST', 'SB', 'AF', 'LBBB', 'RBBB', 'ST', 'SB', 'AF', 'LBBB', 'RBBB',
    'ST', 'ST', 'SB', 'AF', 'LBBB', 'RBBB', 'SB', 'AF', 'LBBB', 'RBBB'
]

# Contar a quantidade de cada tipo
contagem = Counter(order)

# Exibir a contagem
for doenca, quantidade in contagem.items():
    print(f"{doenca}: {quantidade}")

# Mapeia os DataFrames para suas respectivas etiquetas
df_list = {
    'SB': SB,
    'ST': ST,
    'AF': AF,
    'RBBB': RBBB,
    'LBBB': LBBB
}

# Inicializa um DataFrame vazio para armazenar o resultado
dados_selecionados = pd.DataFrame()

# Itera sobre a ordem e adiciona uma linha de cada DataFrame
for name in order:
    if name in df_list and not df_list[name].empty:
        # Remove o primeiro índice (linha) do DataFrame atual
        row = df_list[name].iloc[0:1].reset_index(drop=True)
        dados_selecionados = pd.concat([dados_selecionados, row], ignore_index=True)
        # Remove a linha adicionada do DataFrame original
        df_list[name] = df_list[name].iloc[1:]

print('Tamanho inicial:', dados_selecionados.shape[0])

# Salvar o DataFrame concatenado
dados_selecionados.to_csv('todas_amostras.csv', index=False)

# Identificar e remover duplicatas (mantendo apenas a primeira ocorrência)
duplicatas = dados_selecionados[dados_selecionados.duplicated(subset='exam_id', keep='first')]

# Remove as duplicatas
dados_selecionados = dados_selecionados.drop_duplicates(subset='exam_id')
print('Tamanho depois de remover duplicatas:', dados_selecionados.shape[0])

# Salvar o DataFrame final sem duplicatas
dados_selecionados.to_csv('todas_amostras.csv', index=False)

concatenacao_padrao = pd.concat([ST, SB, AF, LBBB, RBBB], ignore_index=False)

# Ordenar os DataFrames pela coluna 'exam_id'
ordenados = dados_selecionados.sort_values(by='exam_id').reset_index(drop=True)
concatenacao_padrao = concatenacao_padrao.sort_values(by='exam_id').reset_index(drop=True)

# Verifica se os DataFrames são iguais
sao_iguais = ordenados.equals(concatenacao_padrao)

print("Os DataFrames são iguais:", sao_iguais)

SB: 33
AF: 28
LBBB: 23
RBBB: 20
ST: 8
Tamanho inicial: 112
Tamanho depois de remover duplicatas: 112
Os DataFrames são iguais: True


In [4]:
import pandas as pd

# Carregar o DataFrame a partir do arquivo CSV
df = pd.read_csv('todas_amostras.csv')

# Contar o número de True em cada coluna
true_counts = df.apply(lambda x: x.eq(True).sum())

# Exibir o número de True em cada coluna
print(true_counts)


exam_id               0
label               112
1dAVb                 0
RBBB                 20
LBBB                 23
SB                   33
ST                    8
AF                   28
normal_ecg            0
silhouette_score      0
tipo                  0
dtype: int64


In [5]:
# SALVAR IDS EM EXCEL 
ids = dados_selecionados['exam_id'].tolist()
ids = pd.DataFrame({
    'exam_id': ids,
    'doenca': [''] * len(ids),
    'borderline': [''] * len(ids),
    'comentario': [''] * len(ids) 
})

num_parts = 1
split_dfs = np.array_split(ids, num_parts)

for i, df_part in enumerate(split_dfs):
    excel_path = f'_planilha_{i+1}.xlsx'
    df_part.to_excel(excel_path, index=False)
    print(f"Arquivo Excel '{excel_path}' salvo com sucesso.")

Arquivo Excel '_planilha_1.xlsx' salvo com sucesso.


c:\Users\luisa\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [6]:
for i in range(1):
    excel_path = f'_planilha_{i+1}.xlsx'

    # Adicionar validação de dados para a coluna 'doenca'
    wb = load_workbook(excel_path)
    ws = wb.active

    # Criar uma lista de opções para a validação de dados na coluna 'doenca'
    doenca_options = ['ST', 'SB', 'AF', 'LBBB', 'RBBB', '1dAVB', 'Outra', 'Nenhuma', 'Interferência']
    doenca_dv = DataValidation(type="list", formula1=f'"{",".join(doenca_options)}"', showDropDown=True)

    # Criar uma lista de opções para a validação de dados na coluna 'borderline'
    borderline_options = ['Sim', 'Não']
    borderline_dv = DataValidation(type="list", formula1=f'"{",".join(borderline_options)}"', showDropDown=True)

    # Adicionar a validação às colunas apropriadas
    for row in range(2, len(ids) + 2):   # Começa na linha 2 para pular o cabeçalho
        doenca_cell = ws[f'B{row}']      # Coluna B para 'doenca'
        borderline_cell = ws[f'C{row}']  # Coluna C para 'borderline'
        
        ws.add_data_validation(doenca_dv)
        ws.add_data_validation(borderline_dv)
        
        doenca_dv.add(doenca_cell)
        borderline_dv.add(borderline_cell)

    # Salvar o arquivo com a validação de dados
    wb.save(excel_path)
    print("Validação de dados adicionada e arquivo salvo com sucesso.")

Validação de dados adicionada e arquivo salvo com sucesso.


In [7]:
for i, df_part in enumerate(split_dfs):
    print(df_part.shape[0])

112


In [8]:
def create_folders(base_path, num_folders):
    for i in range(num_folders):
        folder_path = os.path.join(base_path, f'parte_{i+1}')
        os.makedirs(folder_path, exist_ok=True)

def plot_and_save_ecgs(split_dfs, arquivos_hdf5, base_path):
    create_folders(base_path, len(split_dfs))

    # ESCOLHA AQUI QUAL ARTE PROCESSAR (0-3)
    i = 0
    ex = 1
    df_part = split_dfs[i]
    folder_path = os.path.join(base_path, f'parte_{i+1}')
    exam_ids_to_plot = df_part['exam_id'].tolist()
    
    for exam_id in exam_ids_to_plot:
        found = False  
        for arquivo in arquivos_hdf5:
            try:
                with h5py.File(arquivo, 'r') as f:
                    exam_ids = np.array(f['exam_id'])
                    exam_index = np.where(exam_ids == exam_id)[0]

                    if len(exam_index) == 0:
                        continue
                    else:
                        exam_index = exam_index[0]
                        exam_tracings = f['tracings'][exam_index]
                        ecg = np.array(exam_tracings).T
                        ecg_plot.plot(ecg, sample_rate=400,
                            lead_index=['DI', 'DII', 'DIII', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'], style='bw')
                        # rm ticks
                        plt.tick_params(
                            axis='both',  # changes apply to the x-axis
                            which='both',  # both major and minor ticks are affected
                            bottom=False,  # ticks along the bottom edge are off
                            top=False,  # ticks along the top edge are off
                            left=False,
                            right=False,
                            labelleft=False,
                            labelbottom=False)  # labels along the bottom edge are off
                        plt.savefig(os.path.join(folder_path, f'{ex}_{exam_id}.png'), dpi=550)
                        plt.close()
                        found = True
                        ex = ex +1
                        break
            except Exception as e:
                print(f"Erro ao processar o arquivo {arquivo} para o exam ID {exam_id}: {e}")

        if not found:
            print(f"Erro: exame ID {exam_id} não encontrado em nenhum dos arquivos.")
        if found:
            print(f"Sucesso: exame ID {exam_id} salvo.")

base_path = 'imagens'

arquivos_hdf5 = [
    "../../Database/filtered_exams_0_1.hdf5", "../../Database/filtered_exams_2_3.hdf5",
    "../../Database/filtered_exams_4_5.hdf5", "../../Database/filtered_exams_6_7.hdf5",
    "../../Database/filtered_exams_8_9.hdf5", "../../Database/filtered_exams_10_11.hdf5",
    "../../Database/filtered_exams_12_13.hdf5", "../../Database/filtered_exams_14_15.hdf5",
    "../../Database/filtered_exams_16_17.hdf5"
]

plot_and_save_ecgs(split_dfs, arquivos_hdf5, base_path)

print("Processo completo!")

Sucesso: exame ID 1501409 salvo.
Sucesso: exame ID 1178266 salvo.
Sucesso: exame ID 1628423 salvo.
Sucesso: exame ID 1191516 salvo.
Sucesso: exame ID 1548151 salvo.
Sucesso: exame ID 922482 salvo.
Sucesso: exame ID 911598 salvo.
Sucesso: exame ID 1847840 salvo.
Sucesso: exame ID 609256 salvo.
Sucesso: exame ID 909325 salvo.
Sucesso: exame ID 1148424 salvo.
Sucesso: exame ID 420273 salvo.
Sucesso: exame ID 2799973 salvo.
Sucesso: exame ID 965401 salvo.
Sucesso: exame ID 1840734 salvo.
Sucesso: exame ID 1378096 salvo.
Sucesso: exame ID 1359191 salvo.
Sucesso: exame ID 1056318 salvo.
Sucesso: exame ID 1129468 salvo.
Sucesso: exame ID 69129 salvo.
Sucesso: exame ID 1190545 salvo.
Sucesso: exame ID 410553 salvo.
Sucesso: exame ID 1032531 salvo.
Sucesso: exame ID 1144077 salvo.
Sucesso: exame ID 1508797 salvo.
Sucesso: exame ID 1171185 salvo.
Sucesso: exame ID 646912 salvo.
Sucesso: exame ID 2759831 salvo.
Sucesso: exame ID 2996677 salvo.
Sucesso: exame ID 908935 salvo.
Sucesso: exame ID 111